<a href="https://colab.research.google.com/github/luisosmx/SQL_Exercises/blob/main/query_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
WITH UnnestedEvents AS (
  SELECT
    id,
    status,
    created,
    updated,
    summary,
    attendees,
    creator,
    start,
  FROM
    `imagine-growth-institute.google_calendar_api.calendar_events`
),

RankedEvents AS (
  SELECT
    id,
    status,
    created,
    updated,
    summary,
    creator,
    start,
    attendee.email,
    attendee.additionalGuests,
    attendee.comment,
    attendee.displayName,
    attendee.organizer AS attendee_organizer,
    attendee.resource,
    attendee.responseStatus,
    attendee.self,
    ROW_NUMBER() OVER (PARTITION BY attendee.email ORDER BY updated DESC) AS row_num
  FROM UnnestedEvents,
  UNNEST(attendees) AS attendee
),

recent_event AS (
  SELECT
  id,
  status,
  created,
  updated,
  summary,
  creator,
  start,
  IFNULL(email, 'None') AS email,
  additionalGuests,
  comment,
  displayName,
  attendee_organizer,
  resource,
  responseStatus,
  self
FROM RankedEvents
WHERE row_num = 1
),

join_list AS (
  SELECT
  c.summary AS calendar_event_summary,
  c.creator AS calendar_event_creator,
  c.start AS calendar_event_star,

  e.id AS expandi_raw_data_id,
  CONCAT(e.first_name, ' ', e.last_name) AS full_name,
  e.email AS expandi_raw_data_email,
  e.company_name AS expandi_raw_data_company_name

FROM
  recent_event c
RIGHT JOIN
  `imagine-growth-institute.google_calendar_api.expandi_raw_data` e
ON
  c.email = e.email
)

SELECT
COUNT(full_name),
calendar_event_summary,
EXTRACT(MONTH FROM TIMESTAMP(calendar_event_star)) AS month_event
FROM join_list
GROUP BY calendar_event_summary, month_event
#WHERE validated_email = 'None'

